# 初始化环境

In [ ]:
import aie

aie.Authenticate()
aie.Initialize()

# 筛选需要下载数据的矢量区域

In [ ]:
# 指定需要检索的区域
shp = aie.FeatureCollection('China_City') \
         .filter(aie.Filter.eq('city', '杭州市'))

map = aie.Map(
    center=shp.getCenter(),
    height=800,
    zoom=8
)

vis_params = {
    'color': '#00FF00'
}

map.addLayer(
    shp,
    vis_params,
    'filter-shp',
    bounds=shp.getBounds()
)
map

In [ ]:
region = shp.geometry()
print(region.getInfo())

# 筛选需要下载的影像集合并进行镶嵌拼接显示

In [ ]:
def s2_mosaic(start_time, end_time):
    s2 = aie.ImageCollection('SENTINEL_MSIL2A') \
            .filterBounds(region) \
            .filterDate(start_time, end_time) \
            .filter('eo:cloud_cover<20') \
            .limit(20) \
            .select(['B4', 'B3', 'B2'])
    result = s2.mean().clip(region)
    return result

image = s2_mosaic('2021-04-01', '2022-01-31')

map = aie.Map(
    center=shp.getCenter(),
    height=800,
    zoom=8
)

vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 200,
    'max': 1800
}
    
map.addLayer(
    image,
    vis_params,
    'S2',
    bounds=shp.getBounds()
)
map

# 创建数据导出任务

In [ ]:
task = aie.Export.image.toAsset(image, 'S2_EXPORT_20220901', 100)
task.start()